# CP Factorization

## Initialization of libraries and Dataset

In [23]:
%matplotlib inline
import os
import sys
import matplotlib
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import pickle
from collections import defaultdict

from datetime import datetime, date
from sklearn.metrics import precision_recall_curve, average_precision_score

from sktensor import dtensor, cp_als

from numpy import linalg as LA

matplotlib.style.use('ggplot')

In [24]:
df = pickle.load( open( "dblp_inproceeding_10.p", "rb" ) )

In [25]:
df_train = df[df.year>=1991]
df_train = df[df.year<=2000]

In [26]:
df_test = df[df.year==2001]

In [27]:
df_train.shape, df_test.shape

((572596, 3), (64233, 3))

### More than 20 publications

In [29]:
author_list = pd.unique(df_train.iloc[:,0])

In [30]:
# Get the indexes of each author

temp = time.time()
count = 0

authors_indexes = [(element, index) for index, element in enumerate(df_train.author)]
author_index = defaultdict(list)

for k, v in authors_indexes:
    if count % 10000 == 1:
        print('%d/%d lines treated - %.0f seconds elapsed') %(count, len(author_list), time.time() - temp)
  
    author_index[k].append(v)
    
    count += 1

1/53454 lines treated - 3 seconds elapsed
10001/53454 lines treated - 3 seconds elapsed
20001/53454 lines treated - 3 seconds elapsed
30001/53454 lines treated - 3 seconds elapsed
40001/53454 lines treated - 3 seconds elapsed
50001/53454 lines treated - 3 seconds elapsed
60001/53454 lines treated - 3 seconds elapsed
70001/53454 lines treated - 3 seconds elapsed
80001/53454 lines treated - 3 seconds elapsed
90001/53454 lines treated - 3 seconds elapsed
100001/53454 lines treated - 3 seconds elapsed
110001/53454 lines treated - 3 seconds elapsed
120001/53454 lines treated - 3 seconds elapsed
130001/53454 lines treated - 3 seconds elapsed
140001/53454 lines treated - 3 seconds elapsed
150001/53454 lines treated - 3 seconds elapsed
160001/53454 lines treated - 3 seconds elapsed
170001/53454 lines treated - 3 seconds elapsed
180001/53454 lines treated - 3 seconds elapsed
190001/53454 lines treated - 3 seconds elapsed
200001/53454 lines treated - 3 seconds elapsed
210001/53454 lines treated 

In [31]:
# Keep the authors with more than 20 publications
author_index_sorted = []

for author in author_list : 
    if len(author_index[author]) > 20 :
        author_index_sorted += author_index[author]

In [32]:
df_train = df_train[df_train.index.isin(author_index_sorted)]

In [33]:
df_test = df_test[df_test.author.isin(df_train.author)]
df_test = df_test[df_test.crossref.isin(df_train.crossref)]

In [34]:
df_train.shape, df_test.shape

((122244, 3), (4821, 3))

In [35]:
del df

### Create the training tensor

In [36]:
author_list = pd.unique(df_train.iloc[:, 0])
conf_list = pd.unique(df_train.iloc[:, 2])
year_list = pd.unique(df_train.iloc[:, 1])

year_list.sort()

author_dic = {element : index for index, element in enumerate(author_list)}
conf_dic =   {element : index for index, element in enumerate(conf_list)}
year_dic =   {element : index for index, element in enumerate(year_list)}

T_train = np.zeros((len(author_list), len(conf_list), len(year_list)))
T_train = dtensor(T_train)

In [37]:
# Going through the dataframe
t_temp = time.time()
i = 0

L = len(zip(df_train.author, df_train.crossref, df_train.year))

for a, c, y in zip(df_train.author, df_train.crossref, df_train.year):
    if i % 100000 == 1:
        print('%d/%d (df.author, df.crossref, df.year) treated - %.0f seconds elapsed') %(i, L, time.time() - t_temp)
        
    # Finding the corresponding index in the tensor
    a_ind = author_dic[a]
    c_ind = conf_dic[c]
    y_ind = year_dic[y]
    
    # Modifying the tensor value for the tuple (i_ind, j_ind, k_ind)
    T_train[a_ind, c_ind, y_ind] += 1
    
    i +=1

print time.time()-t_temp    

1/122244 (df.author, df.crossref, df.year) treated - 0 seconds elapsed
100001/122244 (df.author, df.crossref, df.year) treated - 1 seconds elapsed
1.04016494751


In [38]:
# Logarithmic Transformation
nonz = T_train.nonzero()
for ind in range(len(nonz[0])):
    i_ind = nonz[0][ind]
    j_ind = nonz[1][ind] 
    k_ind = nonz[2][ind]
    
    T_train[i_ind, j_ind, k_ind] = 1 + np.log(T_train[i_ind, j_ind, k_ind]) 

In [39]:
T_train.shape

(39827, 1457, 10)

### Create the test tensor

In [40]:
author_list_train = author_list
conf_list_train = conf_list
author_dic_train = author_dic
conf_dic_train = conf_dic

In [41]:
author_list = pd.unique(df_test.iloc[:, 0])
conf_list = pd.unique(df_test.iloc[:, 2])

author_dic = {element : index for index, element in enumerate(author_list)}
conf_dic =   {element : index for index, element in enumerate(conf_list)}

T_test = np.zeros((len(author_list_train), len(conf_list_train)))

In [42]:
# Going through the dataframe
t_temp = time.time()
i = 0

L = len(zip(df_test.author, df_test.crossref))

for a, c in zip(df_test.author, df_test.crossref):
    if i % 1000 == 1:
        print('%d/%d (df.author, df.crossref, df.year) treated - %.0f seconds elapsed') %(i, L, time.time() - t_temp)

    # Finding the corresponding index in the tensor
    a_ind = author_dic_train[a]
    c_ind = conf_dic_train[c]

    # Modifying the tensor value for the tuple (i_ind, j_ind, k_ind)
    T_test[a_ind, c_ind] = 1

    i +=1

print time.time()-t_temp    

1/4821 (df.author, df.crossref, df.year) treated - 0 seconds elapsed
1001/4821 (df.author, df.crossref, df.year) treated - 0 seconds elapsed
2001/4821 (df.author, df.crossref, df.year) treated - 0 seconds elapsed
3001/4821 (df.author, df.crossref, df.year) treated - 0 seconds elapsed
4001/4821 (df.author, df.crossref, df.year) treated - 0 seconds elapsed
0.324599027634


In [43]:
del author_list_train, conf_list_train, author_list, conf_list, author_dic, conf_dic

## Tensor Factorization

In [44]:
#Proportion of non-zero entries of the tensor
mail_rate = np.count_nonzero(T_train) / float(T_train.shape[0] * T_train.shape[1] * T_train.shape[2])

In [45]:
mail_rate

0.00011316962334299

### CanDecom/ParaFac (CP) Decomposition

In [46]:
# Defining a function for the outer product of several vectors
def Outer(vecs):
    # vecs can be either a list of vectors (n vectors of dimensions (**,1)) or an array of dimensions (**, n)
    vec_list = []
    if isinstance(vecs, list):
        vec_list = vecs
    elif isinstance(vecs, np.ndarray):
        vec_list = [vecs[:, j] for j in range(vecs.shape[1])]        

    res = reduce(np.multiply, np.ix_(*vec_list))
    return res

In [80]:
K = 3 # rank parameter of the CP
%time P, fit, itr, exectimes = cp_als(T_train, K, init='random')
A, B, C = P.U

KeyboardInterrupt: 

In [48]:
# Two equivalent ways of computing the approximation of T_train

#T_approx1 = P.totensor()
#T_approx2 = sum(map(lambda x: P.lmbda[x]*Outer([A[:, x],
#                                                B[:, x],
#                                                C[:, x]]), range(K)))

#np.allclose(T_approx1, T_approx2)

In [15]:
del T_approx1, T_approx2

### CP Scoring

In [33]:
# Collapsed Unweighted Tensor [P04]
tau = 2
gamma = sum(C[-tau:, :], 0)  # The temporal coefficient

S = sum(map(lambda x: gamma[x]*P.lmbda[x]*Outer([A[:, x],
                                                 B[:, x]]), range(K)))
print('Predicting time: %s \nTraining window: %d to %d \nTau_Window: %d dates from %d to %d' %(int(year_list[t_1][:4]),
                                                                                               int(year_list[t_0][:4]),
                                                                                               int(year_list[t_1-1][:4]),
                                                                                               tau,
                                                                                               int(year_list[t_1-tau][:4]),
                                                                                               int(year_list[t_1-1][:4])))

IndexError: index 18 is out of bounds for axis 0 with size 16

In [19]:
del A, B, C

In [51]:
#Converting S into S_pred, a binary matrix
#thres = 1  # If the score above thres, we predict an email

#S_pred = (1*(S >= thres))
#S_test = (1*np.array(T_test[:, :] >= 1))

In [21]:
S.min()

-1.6857733187223011e-07

In [22]:
S.max()

0.00019981750240964961

## Scoring Analysis

### Maximal prediction

In [30]:
# Which pair of (sender, receiver) has a maximal score? 
i_ind, j_ind = np.unravel_index(S.argmax(), S.shape)
i, j = author_list[i_ind], conf_list[j_ind]
print('Predicted Score for author %d to coauthor %d for time %d: %d publications'%(i,
                                                                                   j,
                                                                                   int(year_list[t_1][:4]),
                                                                                   round(S[i_ind, j_ind])))

NameError: name 'i_list' is not defined

In [ ]:
X_max = X.ix[(X.author == i)&(X.coauthor == j)&(X.datetime <= k_list[t_1])]

nb_train = len(X_max.ix[X.datetime < k_list[t_1]])
nb_train_tau = len(X_max.ix[(X.datetime < k_list[t_1])&(X.datetime >= k_list[t_1-tau])])
nb_test = len(X_max.ix[X.datetime == k_list[t_1]])

print('%d publications written during the trainset with %d during the tau-selected window. \n%d publication actually written at time %d' \
      %(nb_train,
        nb_train_tau,
        nb_test,
        k_list[t_1]))

In [ ]:
X_max_count = X_max.datetime.groupby(X_max.datetime).count()

plt.figure(figsize=(16, 10))
plt.title('Temporal Distribution per day \n Sender %d to Receiver %d ' %(i, j))
plt.ylabel('Number of mails')
plt.xlabel('datetime')
plt.xlim([k_list[t_0], k_list[t_1]+10])
plt.grid('on')
plt.plot(X_max_count.index, X_max_count)
plt.xticks(np.arange(k_list[t_0], k_list[t_1]+10, 5), rotation=90)
plt.ticklabel_format(useOffset=False)

plt.axvline(k_list[t_1-tau])
plt.axvline(k_list[t_1-1])

### Factorization Analysis

In [ ]:
for k in (-P.lmbda).argsort()[:5]:
    i_ind = abs(A[:, k]).argmax()
    i_max = i_list[i_ind]

    j_ind = abs(B[:, k]).argmax()
    j_max = j_list[j_ind]

    k_ind = abs(C[:, k]).argmax()
    k_max = k_list[k_ind]

    plt.figure(figsize=(8, 5))
    plt.title('Sender coefficient from component %d \nSender_max = %d' %((k+1), i_max))
    plt.ylabel('Absolute value of coefficient')
    plt.xlabel('Sender index')
    plt.plot(A[:, k])

    plt.figure(figsize=(8, 5))
    plt.title('Receiver coefficient from component %d \nReceiver_max = %d' %((k+1), j_max))
    plt.ylabel('Absolute value of coefficient')
    plt.xlabel('Receiver index')
    plt.plot(B[:, k])

    plt.figure(figsize=(8, 5))
    plt.title('Time coefficient from component %d \nTime_max = %d \n Lambda = %f' %((k+1), k_max, P.lmbda[k]))
    plt.ylabel('Absolute value of coefficient')
    plt.xlabel('Time index')
    plt.plot(C[:, k])

### Precision-Recall for several values of K
Collapsed Unweighted Tensor

In [52]:
#K_list = [1, 5, 10, 50]
K_list = [3]
#tau_list = [1, 7, 50]
tau = 2
S = np.zeros((T_train.shape[0], T_train.shape[1]))

# Initialization
precision = 0
recall = 0
average_precision = dict()
for K_ind in range(len(K_list)):
    # Choice of K
    K = K_list[K_ind]
    # CP Decomposition
    P, fit, itr, exectimes = cp_als(T_train, K, init='random')
    A, B, C = P.U
    # CP Scoring
    gamma = sum(C[-tau:, :], 0)  # The temporal coefficient
    Sk = sum(map(lambda x: gamma[x]*P.lmbda[x]*Outer([A[:, x], B[:, x]]), range(K)))
    S += Sk/LA.norm(Sk, 'fro')

y_score = S.flatten()
y_test = T_test.flatten()    

from sklearn import metrics
ACCURACY = metrics.accuracy_score(T_test, S)
   

ValueError: Can't handle mix of multilabel-indicator and continuous-multioutput

In [ ]:
# Precision-Recall
precision, recall, _ = precision_recall_curve(y_test, y_score)
average_precision = average_precision_score(y_test, y_score)    

# Plotting the results
plt.clf()
plt.figure(figsize=(16, 10))

plt.plot(recall, precision, label='AUC={0:0.2f}' .format(average_precision))

plt.axhline(mail_rate, label='random', color='black')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.legend(loc="upper right")
plt.title('Precision-Recall for CP Factorization of rank K \nCollapsed Unweighted Tensor -  Tau = %d days' %(tau))
plt.savefig('cp_roc_cut_%d.png' %(tau)) 

### Precision-Recall for several values of K
Exponential Smoothing

In [ ]:
del A, B, C

In [ ]:
#K_list = [1, 5, 10, 50]
K_list = [3]
#tau_list = [1, 7, 50]
tau = 2
#alpha_list = [0.2, 0.5, 0.8]
alpha = 0.2
S = np.zeros((T_train.shape[0], T_train.shape[1]))

# Initialization
precision = 0
recall = 0
average_precision = 0

for K_ind in range(len(K_list)):
    # Choice of K
    K = K_list[K_ind]
    # CP Decomposition
    P, fit, itr, exectimes = cp_als(T_train, K, init='random')
    A, B, C = P.U
    # CP Scoring
    gamma = sum(map(lambda x: ((1 - alpha) ** x) * alpha * C[-x, :], range(t_1 - t_0 - 1)), 0) + \
            (1 - alpha) ** (t_1 - t_0 + 1) * C[0, :]                                 # Exponential smoothing
    Sk = sum(map(lambda x: gamma[x]*P.lmbda[x]*Outer([A[:, x], B[:, x]]), range(K)))
    S += Sk/LA.norm(Sk, 'fro')

y_score = S.flatten()
y_test = S_test.flatten()  
    
# Precision-Recall
precision, recall, _ = precision_recall_curve(y_test, y_score)
average_precision = average_precision_score(y_test, y_score)    
    
# Plotting the results
plt.clf()
plt.figure(figsize=(16, 10))

plt.plot(recall, precision, label='AUC={0:0.2f}' .format(average_precision))

plt.axhline(mail_rate, label='random', color='black')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.legend(loc="upper right")
plt.title('Precision-Recall for CP Factorization of rank K \
          \nCollapsed Weighted Tensor -  Alpha = %.2f - Tau = %d days' %(alpha, tau))
plt.savefig('cp_roc_cwt_%.2f_%d.png' %(alpha, tau))    